In [1]:
### NBA DRAFT ANALYSIS

In [1]:
1955 joueurs sans stats du collège dont :
- 1126 joueurs draftés avant 1997 année avant laquelle le site de la NBA n'a pas de stats collège
- 286 joueurs sans profil NBA -> joueurs ayant été draftés mais sans aucune minute en NBA
- 543 joueurs draftés à partir de 1997 qui n'ont pas de stats collège (draftés directement depuis High School ou joueurs internationaux)

SyntaxError: invalid syntax (<ipython-input-1-3bf1c1c0f910>, line 1)

In [2]:
# Importation des librairies utilisées
import pandas as pd
from dateutil.parser import parse
from datetime import timedelta
import numpy as np
import csv
import re

In [3]:
# Sauvegarde des données CSV dans un dataframe pandas
df = pd.read_csv("Webscrapping_Draft_NBA.csv")
print(df)

        ID             Player Name      Drafting Team             NBA profile  \
0        1              Bob Lanier    Detroit Pistons   /stats/player/600005/   
1        2        Rudy Tomjanovich  San Diego Rockets    /stats/player/78350/   
2        3           Pete Maravich      Atlanta Hawks    /stats/player/77459/   
3        4             Dave Cowens     Boston Celtics    /stats/player/76462/   
4        5               Sam Lacey  Cincinnati Royals    /stats/player/77326/   
...    ...                     ...                ...                     ...   
2495  2496     Talen Horton-Tucker      Orlando Magic  /stats/player/1629659/   
2496  2497        Ignas Brazdeikis   Sacramento Kings  /stats/player/1629649/   
2497  2498            Terance Mann        LA Clippers  /stats/player/1629611/   
2498  2499  Quinndary Weatherspoon  San Antonio Spurs  /stats/player/1629683/   
2499  2500        Jarrell Brantley     Indiana Pacers  /stats/player/1629714/   

      College / High School

In [7]:
missing_college = df[df["CO GP"] == 0]

In [8]:
# Saving to CSV
missing_college.to_csv("Missing_college_2.csv")

In [45]:
df4 = pd.read_csv("Webscrapping_Draft_NBA.csv")
df5 = pd.read_csv("Webscrapping_Draft_NBA2.csv")

In [30]:
# Added a ID column to the draft_nba csv therefore reloading it
df = pd.read_csv("Draft_NBA_v2.csv")
df2 = pd.read_csv("Completed_draft.csv")
df3 = pd.read_csv("Webscrapping_college.csv")

In [64]:
print(df["MVP Awards"][df["MVP Awards"] != '0'].count())
print(df["All-NBA Selections"][df["All-NBA Selections"] != '0'].count())

223
96


In [65]:
print(df5["MVP Awards"][df5["MVP Awards"] != '0'].count())
print(df5["All-NBA Selections"][df5["All-NBA Selections"] != '0'].count())

235
95


In [53]:
df4_y = df4[df4["Draft Year"] < 2020]
df4_y["GP"].sum()

856245.0

In [54]:
df5_y = df5[df5["Draft Year"] < 2020]
df5_y["GP"].sum()

856364.0

In [13]:
# Removing first column of df2 which is useless to get the same structure for both dfs
df2.drop("Line", inplace = True, axis = 1)

In [14]:
# ID is a string in the first df, an int in the second -> conversion needed
df["ID"] = pd.to_numeric(df["ID"])
df2["ID"] = pd.to_numeric(df2["ID"])

In [15]:
print(df["ID"][df["Position"] == "Pos Missing"].count())
print(df["ID"][df["Height"] == "(0.00m)"].count())
print(df["ID"][df["Height"] == "Height unknown"].count())

664
74
286


In [16]:
print(df2["ID"][df2["Height"] == "(0.00m)"].count())
print(df2["ID"][df2["Height"] == "Height unknown"].count())
print(df2["ID"][df2["Height"] == "Height still missing"].count())

4
28
0


In [17]:
df["ID"][df["CO GP"] == 0].count()

1955

In [18]:
# Cleaning the CO GP column as the reason why the player has not played any game is indicated as a string
number_rgx = "[0-9]+"

# Looping through the column
for i in range(0, df2["CO GP"].count()):
    # If the element doesn't match the number regex, replace it with 0
    if re.search(number_rgx, df2.iloc[i, 36]) == None:
        df2.iat[i, 36] = 0

In [19]:
# Cleaning the CO APG column as some strings have spaces within which prevents the conversion to float type (ex: "3 .1")
for i in range(0, df2["CO APG"].count()):
    # Keeping only strings
    if type(df2.iloc[i, 43]) != float:
        # Checking if a space is present in it
        if " " in df2.iloc[i, 43]:
            # Removing it from the string
            df2.iat[i, 43] = df2.iloc[i, 43].replace(" ", "")


In [20]:
# Formatting the CO 3PT% column as some strings are empty and can't be converted to float
for i in range(0, df2["CO 3P%"].count()):
    if (df2.iloc[i, 40] == " "):
        df2.iat[i, 40] = "0.0"

In [21]:
# Formatting columns as some strings are composed of --- and can't be converted to float
for i in range(0, df2["ID"].count()):
    # CO BPG column
    if df2.iloc[i, 45] == "---":
        df2.iat[i, 45] = "0.0"
    # CO PPG column
    if df2.iloc[i, 38] == "---":
        df2.iat[i, 38] = "0.0"
    # CO FG%
    if df2.iloc[i, 39] == "---":
        df2.iat[i, 39] = "0.0"
    # CO 3P%
    if df2.iloc[i, 40] == "---":
        df2.iat[i, 40] = "0.0"
    # CO FT%
    if df2.iloc[i, 41] == "---":
        df2.iat[i, 41] = "0.0"
    # CO RPG
    if df2.iloc[i, 42] == "---":
        df2.iat[i, 42] = "0.0"
    # CO APG
    if df2.iloc[i, 43] == "---":
        df2.iat[i, 43] = "0.0"
    # CO SPG
    if df2.iloc[i, 44] == "---":
        df2.iat[i, 44] = "0.0"

In [22]:
# Creating list of zeros matching the size of the first df
zeros = [0] * 2500
# Adding year before draft league to the fisrt df
df["League Before Draft"] = zeros
# Converting column to string type
df["League Before Draft"] = df["League Before Draft"].astype(str)

In [23]:
# Updating the first df with the values retrieved by the 2nd webscrapping
# Looping through all records from 2nd df
for i in range(0, 1955):
    # Find the player in the first df ang get his index
    index = df.loc[df["ID"] == df2.iloc[i, 0]].index[0]
    # If no NBA profile, updating value with college profile link
    if df.iloc[index, 3] == "No NBA profile":
        df.iat[index, 3] = df2.iloc[i, 3]
    # If position is missing in the first df but present in the second, replace value in 1st
    if df.iloc[index, 7] == "Pos Missing" and df2.iloc[i, 7] != "Pos Missing":
        df.iat[index, 7] = df2.iloc[i, 7]
    # Same for height
    if (df.iloc[index, 10] == "Height unknown" or df.iloc[index, 10] == "(0.00m)") and (df2.iloc[i, 10] != "Height unknown" and df2.iloc[i, 10] != "Height still missing" and df2.iloc[i, 10] != "(0.00m)"):
        df.iat[index, 10] = df2.iloc[i, 10]
    # Same for weight
    if (df.iloc[index, 11] == "Weight unknown" or df.iloc[index, 11] == "(0kg)") and (df2.iloc[i, 11] != "Weight unknown" and df2.iloc[i, 11] != "Weight still missing" and df2.iloc[i, 11] != "(0kg)"):
        df.iat[index, 11] = df2.iloc[i, 11]
    # Same for nationality
    if df.iloc[index, 8] == "Nationality unknown" and df2.iloc[i, 8] != "Nationality unknown":
        df.iat[index, 8] = df2.iloc[i, 8]
    # Replacing all college stats if found in df2
    # Games played
    if df2.iloc[i, 36] != 0:
        df.iat[index, 36] = df2.iloc[i, 36]
    # Minutes per game
    if df2.iloc[i, 37] != 0:
        df.iat[index, 37] = df2.iloc[i, 37]
    # Points per game
    if df2.iloc[i, 38] != 0:
        df.iat[index, 38] = df2.iloc[i, 38]
    # Field goal %
    if df2.iloc[i, 39] != 0:
        df.iat[index, 39] = df2.iloc[i, 39]
    # 3pt %
    if df2.iloc[i, 40] != 0:
        df.iat[index, 40] = df2.iloc[i, 40]
    # Free-throw %
    if df2.iloc[i, 41] != 0:
        df.iat[index, 41] = df2.iloc[i, 41]
    # Rebounds per game
    if df2.iloc[i, 42] != 0:
        df.iat[index, 42] = df2.iloc[i, 42]
    # Assists per game
    if df2.iloc[i, 43] != 0:
        df.iat[index, 43] = df2.iloc[i, 43]
    # Steals per game
    if df2.iloc[i, 44] != 0:
        df.iat[index, 44] = df2.iloc[i, 44]
    # Blocks per game
    if df2.iloc[i, 45] != 0:
        df.iat[index, 45] = df2.iloc[i, 45]
    # Turnovers per game
    if df2.iloc[i, 46] != 0:
        df.iat[index, 46] = df2.iloc[i, 46]
    # League before draft
    if df2.iloc[i, 52] != "":
        df.iat[index, 52] = df2.iloc[i, 52]

In [27]:
df["League Before Draft"][df["League Before Draft"] == ""].count()

0

In [32]:
df3["League before Draft"][df3["League before Draft"] == " "].count()

0

In [26]:
df3["CO GP"].isna().sum()

163

In [31]:
# Saving to CSV
df.to_csv("Draft_NBA_semi_completed.csv")

In [157]:
missing_stats = df[df["CO GP"] == 0]
# Saving to CSV
missing_stats.to_csv("Missing_stats.csv")

In [1]:
# Saving hand completed CSV to df
df3 = pd.read_csv("Missing_stats_hand_completion.csv")

NameError: name 'pd' is not defined

In [26]:
# Removing first column of df3 which is useless, to get the same structure for all dfs
df3.drop("Line", inplace = True, axis = 1)

In [32]:
# Updating the first df with the values retrieved in the 3rd df
# Looping through all records from 3rd df
for i in range(0, 225):
    # Find the player in the first df ang get his index
    index = df.loc[df["ID"] == df3.iloc[i, 0]].index[0]
    # If no NBA profile, updating value with college profile link
    if df.iloc[index, 3] == "No NBA profile":
        df.iat[index, 3] = df3.iloc[i, 3]
    # If position is missing in the first df but present in the second, replace value in 1st
    if df.iloc[index, 7] == "Pos Missing" and df3.iloc[i, 7] != "Pos Missing":
        df.iat[index, 7] = df3.iloc[i, 7]
    # Same for height
    if (df.iloc[index, 10] == "Height unknown" or df.iloc[index, 10] == "(0.00m)") and (df3.iloc[i, 10] != "Height unknown" and df3.iloc[i, 10] != "Height still missing" and df3.iloc[i, 10] != "(0.00m)"):
        df.iat[index, 10] = df3.iloc[i, 10]
    # Same for weight
    if (df.iloc[index, 11] == "Weight unknown" or df.iloc[index, 11] == "(0kg)") and (df3.iloc[i, 11] != "Weight unknown" and df3.iloc[i, 11] != "Weight still missing" and df3.iloc[i, 11] != "(0kg)"):
        df.iat[index, 11] = df3.iloc[i, 11]
    # Same for nationality
    if df.iloc[index, 8] == "Nationality unknown" and df3.iloc[i, 8] != "Nationality unknown":
        df.iat[index, 8] = df3.iloc[i, 8]
    # Replacing all college stats if found in df2
    # Games played
    if df3.iloc[i, 36] != 0:
        df.iat[index, 36] = df3.iloc[i, 36]
    # Minutes per game
    if df3.iloc[i, 37] != 0:
        df.iat[index, 37] = df3.iloc[i, 37]
    # Points per game
    if df3.iloc[i, 38] != 0:
        df.iat[index, 38] = df3.iloc[i, 38]
    # Field goal %
    if df3.iloc[i, 39] != 0:
        df.iat[index, 39] = df3.iloc[i, 39]
    # 3pt %
    if df3.iloc[i, 40] != 0:
        df.iat[index, 40] = df3.iloc[i, 40]
    # Free-throw %
    if df3.iloc[i, 41] != 0:
        df.iat[index, 41] = df3.iloc[i, 41]
    # Rebounds per game
    if df3.iloc[i, 42] != 0:
        df.iat[index, 42] = df3.iloc[i, 42]
    # Assists per game
    if df3.iloc[i, 43] != 0:
        df.iat[index, 43] = df3.iloc[i, 43]
    # Steals per game
    if df3.iloc[i, 44] != 0:
        df.iat[index, 44] = df3.iloc[i, 44]
    # Blocks per game
    if df3.iloc[i, 45] != 0:
        df.iat[index, 45] = df3.iloc[i, 45]
    # Turnovers per game
    if df3.iloc[i, 46] != 0:
        df.iat[index, 46] = df3.iloc[i, 46]
    # League before draft
    if df3.iloc[i, 52] != "":
        df.iat[index, 52] = df3.iloc[i, 52]

ValueError: could not convert string to float: 'True 0.0'

In [64]:
# Importing the completed NBA draft csv
df = pd.read_csv("Webscrapping_college_hand_completion.csv")

In [65]:
# Formatting the position column
# Creating empty list to store formatted strings
position_list= []

for i in df["Position"]:
    # Checking if '|' present in string, if true remove it + whitespace following it
    if '|' in i:
        i = i[2:]
    # Generalization of position
    if i == "Shooting Guard" or i == "Point Guard":
        i = "Guard"
    elif i == "Small Forward" or i == "Power Forward":
        i = "Forward"
    elif i == "Point Guard/Shooting Guard" or i == "Shooting Guard/Point Guard":
        i = "Guard"
    elif i == "Small Forward/Power Forward" or i == "Power Forward/Small Forward":
        i = "Forward"
    elif i == "Small Forward/Shooting Guard" or i == "Shooting Guard/Small Forward":
        i = "Forward-Guard"
    elif i == "Center/Power Forward" or i == "Power Forward/Center":
        i = "Center-Forward"
    elif i == "Guard-Forward":
        i = "Forward-Guard"
    elif i == "Forward-Center":
        i = "Center-Forward"
    elif i == "Shooting Guaard":
        i = "Guard"
    
    position_list.append(i)

# Assigning the new list values to the df position column
df["Position"] = position_list


In [66]:
# Converting the string elements from the Birthdate column to date type
# Creating empty list to store the converted values
birthdate_list = []

for i in df["Birthdate"]:
    # Checking type of data : if value present, type = string / if value missing, type = float
    if type(i) == float:
        pass
    elif i == "Birthdate unknown":
        pass
    else:
        # Handling possibility that the date of death is indicated, in which case we want to remove it
        if '-' in i:
            split = i.split()
            i = split[0]
        # Parsing the string via the dateutil library to get a date
        i = parse(i)
        # Only keeping the date, getting rid of time
        i = i.date()
    # Adding to list
    birthdate_list.append(i)

# Assigning the new list values to the df birthdate column
df["Birthdate"] = birthdate_list

            

In [67]:
# Moving and formatting the height & weight columns
# Popping them in order to reinsert them at the desired position later
height = df.pop("Height")
weight = df.pop("Weight")
df.insert(8, "Height", height)
df.insert(9, "Weight", weight)
# Formatting the strings to convert them in float
# Formatting height
# Creating empty list that will store the values
height_list = []
weight_list = []
for i in df["Height"]:
    # Normal string should be of length 7 with the following format (0.00m)
    if len(i) != 7:
        # Removing leading and ending whitespaces from the string
        i = i.strip()
    # Checking for an abnormal format     
    if " " in i:
        # If whitespace present in the string, removing it
        i = i.replace(" ", "")
        # Checking for a missing parenthesis, adding it if true
        if i[0] != '(':
            i = '(' + i
        # Checking for a missing dot, adding it if true
        if '.' not in i:
            i = i[:2] + '.' + i[2:]
        # Checking for a missing m, adding it if true   
        if 'm' not in i:
            i = i[:5] + 'm' + i[5:]
            
    # Removing parenthesis and the meter symbol
    i = i[1:5]
    # Converting the remaining string to float
    i = float(i)
    # Displaying two decimals
    i = "%.2f" % i
    # Adding to list
    height_list.append(i)
    
# Assigning the new list values to the df height column
df["Height"] = height_list


# Formatting weight
for j in df["Weight"]:
    # Checking type of data : if value present, type = string / if value missing, type = float
    if type(j) == float:
        pass
    else:
        # If whitespace present in the string, removing it
        j = j.replace(" ", "")
        # Normal string should be of length 6 or 7 with the following formats (00kg) or (000kg)
        if len(j) != 6 and len(j) != 7:
            # Checking for a missing parenthesis, adding it if true
            if j[0] != '(':
                j = '(' + j
        # Removing opening parenthesis
        j = j[1:]
        # Removing kg symbol and closing parenthesis
        j = j[:-3]
        # Converting string to int
        j = int(j)
    # Adding to list
    weight_list.append(j)

# Assigning the new list values to the df weight column
df["Weight"] = weight_list


In [68]:
# Moving the "League before Draft column"
league = df.pop("League before Draft")
df.insert(5, "League before Draft", league)

In [69]:
# Fixing the nationality data, first webscrapping gives the country but the second indicates nationality : convert everything to country as first webscrapping gives more data
# Creating empty list that will store the values
nationality_list = []

for i in df["Nationality"]:
    if i == "American":
        i = "USA"
    elif i == "Italian":
        i = "Italy"
    elif i == "Jamaican":
        i = "Jamaica"
    elif i == "Dominican":
        i = "Dominican Republic"
    elif i == "Russian":
        i = "Russia"
    elif i == "Canadian":
        i = "Canada"
    elif i == "Israeli":
        i = "Israel"
    elif i == "Virgin Islander":
        i = "US Virgin Islands"
    elif i == "Dutch":
        i = "Netherlands"
    elif i == "British":
        i = "United Kingdom"
    elif i == "French":
        i = "France"
    elif i == "Croatian":
        i = "Croatia"
    elif i == "German":
        i = "Germany"
    elif i == "Argentine":
        i = "Argentina"
    elif i == "Greek":
        i = "Greece"
    elif i == "Colombian":
        i = "Colombia"
    elif i == "Polish":
        i = "Poland"
    elif i == "Spanish":
        i = "Spain"
    elif i == "Georgian":
        i = "Georgia"
    elif i == "Ukrainian":
        i = "Ukraine"
    elif i == "Puerto Rican":
        i = "Puerto Rico"
    elif i == "Belarusian":
        i = "Belarus"
    elif i == "Finnish":
        i = "Finland"
    elif i == "Lithuanian":
        i = "Lithuania"
    elif i == "Bosnian":
        i = "Bosnia and Herzegovina"
    elif i == "Brazilian":
        i = "Brazil"
    elif i == "Turkish":
        i = "Turkey"
    elif i == "Montenegrin":
        i = "Montenegro"
    elif i == "Serbian":
        i = "Serbia"
    elif i == "Swedish":
        i = "Sweden"
    elif i == "Bosnia":
        i = "Bosnia and Herzegovina"
    elif i == "Congo":
        i = "DRC"
    elif i == "Argentine/Spanish":
        i = "Argentina"
    
    
    # Adding to list
    nationality_list.append(i)

# Assigning the new list to the df nationality column
df["Nationality"] = nationality_list
    

In [70]:
# Handling awards columns to make each award match its designated column
# Getting location of each award column
mvp = df.columns.get_loc("MVP Awards")
all_nba = df.columns.get_loc("All-NBA Selections")
all_defensive = df.columns.get_loc("All-Defensive Selections")
finals_mvp = df.columns.get_loc("Finals MVP Awards")
hof = df.columns.get_loc("Hall of Fame Inductee")

for i in range(0, len(df["ID"])):
    mvp_award = 0
    all_nba_sel = 0
    all_defensive_sel = 0
    finals_mvp_award = 0
    hof_inductee = 0
    # Checking first award column
    if df.iloc[i, mvp] != '0':
        award = df.iloc[i, mvp]
        # Checking for corresponding award
        if "NBA Most Valuable Player" in award:
            # Keeping only number of times won
            mvp_award = int(award[0])
        elif "All-NBA" in award:
            all_nba_sel = int(award[:2])
        elif "All-Defensive" in award:
            all_defensive_sel = int(award[:2])
        elif "NBA Finals" in award:
            finals_mvp_award = int(award[0])
        elif "Hall of Fame" in award:
            hof_inductee = 1
        # If award found, check the next column
        if df.iloc[i, all_nba] != '0':
            award = df.iloc[i, all_nba]
            if "NBA Most Valuable Player" in award:
                # Keeping only number of times won
                mvp_award = int(award[0])
            elif "All-NBA" in award:
                all_nba_sel = int(award[:2])
            elif "All-Defensive" in award:
                all_defensive_sel = int(award[:2])
            elif "NBA Finals" in award:
                finals_mvp_award = int(award[0])
            elif "Hall of Fame" in award:
                hof_inductee = 1
            # If award found, check the next column
            if df.iloc[i, all_defensive] != '0':
                award = df.iloc[i, all_defensive]
                if "NBA Most Valuable Player" in award:
                    # Keeping only number of times won
                    mvp_award = int(award[0])
                elif "All-NBA" in award:
                    all_nba_sel = int(award[:2])
                elif "All-Defensive" in award:
                    all_defensive_sel = int(award[2:])
                elif "NBA Finals" in award:
                    finals_mvp_award = int(award[0])
                elif "Hall of Fame" in award:
                    hof_inductee = 1
                # If award found, check the next column
                if df.iloc[i, finals_mvp] != '0':
                    award = df.iloc[i, finals_mvp]
                    if "NBA Most Valuable Player" in award:
                        # Keeping only number of times won
                        mvp_award = int(award[0])
                    elif "All-NBA" in award:
                        all_nba_sel = int(award[:2])
                    elif "All-Defensive" in award:
                        all_defensive_sel = int(award[:2])
                    elif "NBA Finals" in award:
                        finals_mvp_award = int(award[0])
                    elif "Hall of Fame" in award:
                        hof_inductee = 1
                    # If award found, check the next column
                    if df.iloc[i, hof] != '0':
                        award = df.iloc[i, hof]
                        if "NBA Most Valuable Player" in award:
                            # Keeping only number of times won
                            mvp_award = int(award[0])
                        elif "All-NBA" in award:
                            all_nba_sel = int(award[:2])
                        elif "All-Defensive" in award:
                            all_defensive_sel = int(award[:2])
                        elif "NBA Finals" in award:
                            finals_mvp_award = int(award[0])
                        elif "Hall of Fame" in award:
                            hof_inductee = 1
                    else:
                        pass
                else:
                    pass
            else:
               pass
        else:
            pass
        # Replacing values in the row
        df.iat[i, mvp] = mvp_award
        df.iat[i, all_nba] = all_nba_sel
        df.iat[i, all_defensive] = all_defensive_sel
        df.iat[i, finals_mvp] = finals_mvp_award
        df.iat[i, hof] = hof_inductee                                             

In [71]:
# Handling NaN and null values
# First webscrapping can retrieve two types of zeros, a "true" one which indicates that the statistic was registered but the player never took a 3pt shot or made a block and a "false" one (noted -0.0) which indicates that the statistic is not available
# As -0 and 0 makes no difference in float type, conversion to string to make the distinction is needed
df.loc[:, "GP":"CO TOPG"] = df.loc[:, "GP":"CO TOPG"].astype(str)
# Replace all "-0.0" strings by NaN
df.loc[:, "GP":"CO TOPG"] = df.loc[:, "GP":"CO TOPG"].replace("-0.0", np.nan)

In [72]:
# Reconverting all game stats to float
# Removing an extra dot in one data of the df which made the conversion fail
df.iat[1856, 55] = df.iloc[1856, 55][:-1]
# Removing a whitespace in one data of the df which made the conversion fail
df.iat[549, 56] = df.iloc[549, 56].replace(" ", "")
# Removing whitespace from empty string
df.loc[:, "GP":"CO TOPG"] = df.loc[:, "GP":"CO TOPG"].replace(' ', '')
# Replacing empty strings by NaN
df.loc[:, "GP":"CO TOPG"] = df.loc[:, "GP":"CO TOPG"].replace('', np.nan)
# Replacing '---' string by NaN
df.loc[:, "GP":"CO TOPG"] = df.loc[:, "GP":"CO TOPG"].replace('---', np.nan)
# Replacing '----' string by NaN
df.loc[:, "GP":"CO TOPG"] = df.loc[:, "GP":"CO TOPG"].replace('----', np.nan)
# Converting all stats to float
df.loc[:, "GP":"CO TOPG"] = df.loc[:, "GP":"CO TOPG"].astype(float)

In [73]:
# Filling NaN values in specific columns
df["GP"].fillna(0.0, inplace = True)
df["PO GP"].fillna(0.0, inplace = True)

In [74]:
# Ajout d'une nouvelle colonne qui indique le nom actuel de l'équipe qui a draft le joueur
# Liste regroupant toutes les équipes actuelles
franchises = ["Utah Jazz", "Los Angeles Lakers", "Los Angeles Clippers", "Phoenix Suns", "Denver Nuggets", "Dallas Mavericks", "San Antonio Spurs", "Houston Rockets", "Sacramento Kings", "New Orleans Pelicans", "Golden State Warriors", "Oklahoma City Thunder", "Portland Trail Blazers", "Memphis Grizzlies", "Minnesota Timberwolves", "Philadelphia 76ers", "Brooklyn Nets", "Milwaukee Bucks", "Miami Heat", "Toronto Raptors", "Boston Celtics", "Atlanta Hawks", "Chicago Bulls", "Cleveland Cavaliers", "Detroit Pistons", "Indiana Pacers", "New York Knicks", "Washington Wizards", "Orlando Magic", "Charlotte Hornets"]
current_franchise = []
for i in range(0, 2500):
    # If drafting franchise is also a current one, add it to new list
    if df.iloc[i, 2] in franchises:
        current_franchise.append(df.iloc[i, 2])
    # Else, replacing it by its current name
    elif df.iloc[i, 2] == "San Diego Rockets":
        current_franchise.append("Houston Rockets")
    elif df.iloc[i, 2] == "Seattle SuperSonics":
        current_franchise.append("Oklahoma City Thunder")
    elif df.iloc[i, 2] == "Baltimore Bullets" or df.iloc[i, 2] == "Capital Bullets" or df.iloc[i, 2] == "Washington Bullets":
        current_franchise.append("Washington Wizards")
    elif df.iloc[i, 2] == "Buffalo Braves" or df.iloc[i, 2] == "San Diego Clippers" or df.iloc[i, 2] == "LA Clippers":
        current_franchise.append("Los Angeles Clippers")
    elif df.iloc[i, 2] == "San Francisco Warriors":
        current_franchise.append("Golden State Warriors")
    elif df.iloc[i, 2] == "Kansas City-Omaha Kings" or df.iloc[i, 2] == "Kansas City Kings" or df.iloc[i, 2] == "Cincinnati Royals":
        current_franchise.append("Sacramento Kings")
    elif df.iloc[i, 2] == "New Orleans Jazz":
        current_franchise.append("Utah Jazz")
    elif df.iloc[i, 2] == "New Jersey Nets":
        current_franchise.append("Brooklyn Nets")
    elif df.iloc[i, 2] == "Vancouver Grizzlies":
        current_franchise.append("Memphis Grizzlies")
    elif df.iloc[i, 2] == "New Orleans Hornets" or df.iloc[i, 2] == "New Orleans/Oklahoma City Hornets":
        current_franchise.append("New Orleans Pelicans")
    elif df.iloc[i, 2] == "Charlotte Bobcats":
        current_franchise.append("Charlotte Hornets")
        
# Adding new column to df
df.insert(3, "Franchise current name", current_franchise, True)   
        

In [75]:
# Adding age at draft column
draft_age = []
for i in range(0, 2500):
    # Adding historical draft date to the draft year
    draft_date = "Jun 30, " + str(df.iloc[i, 7])
    # Converting it to datetime type
    draft_date = parse(draft_date)
    # Keeping only the date
    draft_date = draft_date.date()
    # Making sure birthdate is known
    if type(df.iloc[i, 13]) != float:
        age = draft_date - df.iloc[i, 13]
        # Result is of timedelta type, converting it to int by dividing by another timedelta value
        age = age / timedelta(days = 1)
        # Result is in days, converting to years
        age = round(age / 365, 1)
        draft_age.append(age)
    else:
        draft_age.append(np.nan)

# Adding new column to df
df.insert(6, "Age at Draft", draft_age)

In [76]:
# Adding continent column
continent = []
europe = ["Spain", "Lithuania", "Greece", "Italy", "Serbia", "Croatia", "Netherlands", "Romania", "Russia", "Ukraine", "Estonia", "France", "Slovenia", "Germany", "United Kingdom", "Bosnia and Herzegovina", "Finland", "Czech Republic", "Scotland", "Montenegro", "Poland", "Latvia", "Switzerland", "Belarus", "Sweden", "Austria", "Bosnia"]
africa = ["Nigeria", "DRC", "Senegal", "Mali", "Gabon", "Republic of the Congo", "Cameroon", "Tanzania", "South Sudan", "Angola", "Sudan", "Cabo Verde"]
america = ["Canada", "US Virgin Islands", "Jamaica", "Dominican Republic", "Colombia", "Argentina", "St. Vincent & Grenadines", "Mexico", "Haiti", "Brazil", "Puerto Rico", "Venezuela", "Bahamas"]
asia = ["Israel", "Turkey", "Georgia", "China", "South Korea", "Japan"]
oceania = ["Australia", "New Zealand"]

for i in df["Nationality"]:
    # Checking if value is different than US which will be a separate value
    if i != "USA":
        if i in europe:
            continent.append("Europe")
        elif i in africa:
            continent.append("Africa")
        elif i in america:
            continent.append("America")
        elif i in asia:
            continent.append("Asia")
        elif i in oceania:
            continent.append("Oceania")
        else:
            print(i)
            continent.append("Unknown country")
    else:
        continent.append("USA")

# Adding new column to df
df.insert(14, "Continent", continent)
        

,ID,Player Name,Drafting Team,Franchise current name,NBA profile,College / High School,Age at Draft,League before Draft,Draft Year,Pick Number,...,CO APG,CO SPG,CO BPG,CO TOPG,MVP Awards,All-NBA Selections,All-Defensive Selections,Finals MVP Awards,Hall of Fame Inductee,Reason no stats
0,1,Bob Lanier,Detroit Pistons,Detroit Pistons,/stats/player/600005/,St. Bonaventure,21.8,College,1970,1,...,NaN,NaN,NaN,NaN,0,0,0,0,1,Too many results
1,2,Rudy Tomjanovich,San Diego Rockets,Houston Rockets,/stats/player/78350/,Michigan,21.6,College,1970,2,...,NaN,NaN,NaN,NaN,0,0,0,0,0,NaN
2,3,Pete Maravich,Atlanta Hawks,Atlanta Hawks,/stats/player/77459/,Louisiana State,23.0,College,1970,3,...,6.2,NaN,NaN,NaN,0,4,0,0,1,NaN
3,4,Dave Cowens,Boston Celtics,Boston Celtics,/stats/player/76462/,Florida State,21.7,College,1970,4,...,NaN,NaN,NaN,NaN,1,3,3,0,1,NaN
4,5,Sam Lacey,Cincinnati Royals,Sacramento Kings,/stats/player/77326/,New Mexico State,22.3,College,1970,5,...,1.2,NaN,NaN,NaN,0,0,0,0,0,NaN


In [79]:
# Saving to final csv
df.to_csv("Completed_NBA_Draft.csv")